# Ask a youtube video

### Install dependencies

In [90]:
#!pip install langchain
#!pip install langchain-community
#!pip install python-dotenv
#!pip install langchain-openai
#!pip install weaviate-client
#!pip install openai
#!pip install youtube_transcript_api

### Load .env variables

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

YOUR_OPENAI_KEY = os.getenv("YOUR_OPENAI_KEY")
YOUR_WEAVIATE_KEY = os.getenv("YOUR_WEAVIATE_KEY")
YOUR_WEAVIATE_URL = os.getenv("YOUR_WEAVIATE_URL")

# 1. Fetch the video information

In [2]:
video_urls = [ "https://youtu.be/9oRDksmH0zM?si=-DSwzjApEfJnPM4y", "https://youtu.be/oSSEcftwRr0?si=yB4xnPMklA6tz4oZ" ]
from youtube_transcript_api import YouTubeTranscriptApi

def get_video_id(youtube_link: str) -> str:

    video_id = youtube_link.split("v=")
    if len(video_id) > 1:
        return video_id[1]
    video_id = youtube_link.split("youtu.be/")
    if len(video_id) > 1:
        return video_id[1]
    
    raise Exception("Unable to find video id")

def fetch_transcript(youtube_link : str) -> str:

    video_id = get_video_id(youtube_link)

    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en','fr'])

    content = ""
    for line in transcript:
        content += line['text'] + " "

    return content

video_transcripts = [ fetch_transcript(video_url) for video_url in video_urls ]

# 2. Split transcript into sentences

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_transcripts = [ text_splitter.split_text(transcript) for transcript in video_transcripts ]

# 3. Vector DB Storage

In [12]:
import weaviate
import weaviate.classes as wvc
from langchain.vectorstores import Weaviate

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=YOUR_WEAVIATE_URL,
    auth_credentials=weaviate.auth.AuthApiKey(api_key=YOUR_WEAVIATE_KEY),  # Replace with your Weaviate instance API key
    headers = {
        "X-OpenAI-Api-Key": YOUR_OPENAI_KEY # Replace with your inference API key
    }
)

print("Client is ready: ", client.is_ready())

client.collections.delete_all()

try:
    # Create the Transcript collection
    collection = client.collections.create(
        name="Transcript",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
            model="text-embedding-3-small"  # Specify the model
        ),
        properties=[
            wvc.config.Property(
                name="text",
                data_type=wvc.config.DataType.TEXT,  # Data type for the property
                description="Text of the transcript"  # Description of the property
            ),
        ]
    )
finally:
    client.close()

client = weaviate.Client(url=YOUR_WEAVIATE_URL, auth_client_secret=weaviate.auth.AuthApiKey(YOUR_WEAVIATE_KEY), additional_headers={"X-OpenAI-Api-Key": YOUR_OPENAI_KEY})
vectorstore = Weaviate(client, "Transcript", "text", attributes=["source"])


Client is ready:  True


In [17]:
# Load text into the vector store

text_meta_pair = []
for transcript, video_url in zip(split_transcripts, video_urls):
    for chunk in transcript:
        text_meta_pair.append((chunk, {"source": video_url}))
texts, metas = zip(*text_meta_pair)
vectorstore.add_texts(texts, metas)

['0668dd12-6d01-47de-9d1a-9dd422fa7dca',
 'efe99d63-67f6-4018-b954-e4e9c1ce41d9',
 '028e71d9-4445-4606-bb02-9e1641eb4cef',
 '06f15409-49b0-47a2-b35c-57e1d21370c8',
 '0ec027e8-224b-45b9-8b80-14c7ebcc4731',
 '1e7a9351-cf7e-48c3-9619-d3214ec08ac1',
 '4ed00658-67a7-4191-9dc9-d2d95aad7ee7',
 '466f9697-f4ca-4a31-958e-4f66661c2112',
 '84224f08-1f37-4079-ae81-a318baee84b0',
 '7d9350fc-f181-4f10-83c1-c3f2b5d0a473',
 '58a41452-dadc-4502-a1fe-c2aedb33a8cc',
 '067be0e9-98da-42d5-8b7e-7169fb759c8f',
 'de6b863e-a3fa-4b34-a32f-d4b12330582a',
 '229f4a8e-8e0d-49ef-970e-50c481170b6a',
 '8b5bc4eb-ad0c-4d37-8cbf-5579fb95aa27',
 'a678ca59-00a7-4a58-8f6a-026f25c1e14c',
 '80467bab-550a-4fdc-b609-78889ab038ff',
 '7d2f10dd-cde0-4ba6-a58e-0570de2a4a73',
 '16d78215-4eb3-43b1-8cb1-92f2ff122547',
 '14eb69a3-719d-4ddf-9b0f-1307537cb865',
 '4af8e259-8856-4a43-a622-88ac96cad412',
 'd5b29843-24b1-4fd2-b864-7c18ec04c57a',
 '07cd733d-072d-4921-9302-276beb321411',
 '370168d6-1fa0-47dd-b0e8-daa0bdda2e9f',
 '2dc95613-4af5-

# 4. Similarity Search

In [22]:
query = "Réponds en français. Donne moi un point commmun entre Apple Intelligence et le ELO aux échecs. "

# retrieve t"ext related to the query
vect = vectorstore.similarity_search(query, top_k=20)
vect

[Document(page_content="on y est Apple franchit enfin le cap de liia et comment le faire d'une meilleure façon que sa proprieré le mot Ia en anglais ai Apple intelligence ouais ils sont extrêmement fort côté marketing Apple ils l'ont fait maintenant liya ça veut dire Apple intelligence ou intelligence Apple ça c'est fort bon en même temps faut dire qu'Apple était quand même à la ramasse sur l'intelligence artificielle depuis des années on le sait Google chat GPT Microsoft et beaucoup d'autres compagnies et sociétés américaines voire mondial si était mis avac pris le lead Apple pourtant en 2011 avait introduit finalement un peu ce qui est devenu l'intelligence artificielle avec un assistant vocal c'était Siri avec l'iPhone 4S un assistant vocal qui était en avance sur son temps en 2011 et qui a été complètement délaissé jusqu'à devenir aujourd'hui peut-être le pire assistant vocal qui existe aujourd'hui parce que tout simplement bah il permet de pas faire grandchose il comprend pas gran

# 5. My Custom Chatbot

In [24]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# define chain
chain = load_qa_chain(
                                            # Initially, temperature param was 0 but 0.1 is better
    OpenAI(openai_api_key = YOUR_OPENAI_KEY, temperature=0.1),
    chain_type="stuff"
)

# create answer
chain.run(input_documents=vect, question=query)

/home/dup/Documents/Pro/InsideBoardAI/AskYourVideo/.venv/lib/python3.12/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


" Un point commun entre Apple Intelligence et le ELO aux échecs est qu'ils sont tous les deux des systèmes de notation qui évaluent le niveau intrinsèque des joueurs ou des utilisateurs."